In [9]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [10]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 3.6 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 


In [11]:
import pandas as pd

from pyomo.environ import *
from pyomo.opt import SolverFactory

In [84]:
#Load data for 200 products from a excel file
products=pd.read_excel(open('MP_scenarios.xlsx', 'rb'),
              sheet_name='Sheet1',index_col=0)
products.head()

,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
No,,,,,,,,
1,55.48,120,160,194,223,15,1421,13.870
2,53.68,115,149,171,197,15,2396,13.420
3,61.56,140,191,207,459,15,2544,15.390
4,65.72,115,151,242,278,15,1316,16.430
5,64.98,120,173,221,335,15,1377,16.245


In [85]:
#Write a function to solve the optimization problem 

def solve(price,demand,weeks,inv,sal):

  n=4 #We are considering 4 different price levels 
  p=[1,0.85,0.7,0.5]

  #Create a model 
  model=ConcreteModel()

  #Declare 4 DVs: 
  model.x=Var(range(n), domain=NonNegativeReals)

  #Declare constraints 
  model.constraint1=Constraint(expr=sum(model.x[i] for i in range(n)) <= weeks )
  model.constraint2=Constraint(expr=sum(model.x[i]*demand[i] for i in range(n)) <= inv)

  #Define Objective
  model.Objective=Objective(expr=sum(model.x[i]*demand[i]*p[i]*price for i in range(n))+ (inv - sum(model.x[i]*demand[i]*sal for i in range(n))), sense=maximize)
  #model.Objective=Objective(expr=sum(model.x[i]*demand[i]*p[i]*price for i in range(4)), sense=maximize) --Without salvage value 
  
  #Solve
  opt = SolverFactory('glpk') 
  results = opt.solve(model)

  #Create a list to show weeks at each price  
  solution=[]
  for i in range(n):
    solution.append(model.x[i]())
  
  solution.append(model.Objective())

  for i in range(n):
   if model.x[i]() != 0: 
    solution.append(p[i]*price)
    break
  
  #Return  
  return solution

In [86]:
#Run a for loop to solve the optimization problem for all 200 products
output=[]
for i in range(len(products)):
  rowdata=products.iloc[i].values.tolist()
  price=rowdata[0]
  demand=rowdata[1:5]
  weeks=rowdata[5]
  inv=rowdata[6]
  sal=rowdata[7]
  output.append(solve(price,demand,weeks,inv,sal))

In [87]:
#Create 3 different lists for each required output measure
num_week=[]
for i in output: 
  num_week.append(i[0:4])

revenue=[]
for i in output: 
  revenue.append(i[4])

current_price=[]
for i in output: 
  current_price.append(i[5])

In [88]:
#Add the 3 lists to products
products["Number of Weeks"]=num_week
products["Projected Revenue"]=revenue
products["Current Price"]=current_price
products.head()

,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value,Number of Weeks,Projected Revenue,Current Price
No,,,,,,,,,,,
1,55.48,120,160,194,223,15,1421,13.870,"[11.8416666666667, 0.0, 0.0, 0.0]",60548.810000,55.480
2,53.68,115,149,171,197,15,2396,13.420,"[0.0, 15.0, 0.0, 0.0]",74380.880000,45.628
3,61.56,140,191,207,459,15,2544,15.390,"[6.29411764705882, 8.70588235294118, 0.0, 0.0]",104645.967529,61.560
4,65.72,115,151,242,278,15,1316,16.430,"[11.4434782608696, 0.0, 0.0, 0.0]",66181.640000,65.720
5,64.98,120,173,221,335,15,1377,16.245,"[11.475, 0.0, 0.0, 0.0]",68485.095000,64.980


In [89]:
#Store as a csv
products.to_csv("P2_Chelsi_Gondalia.csv",index=False)